In [58]:
from tqdm import tqdm
import math
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import json
import re
import spacy
sp = spacy.load('en_core_web_sm')
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import statistics

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/oliverwang/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
f = open('./labelled_sentences.json')
data = json.load(f)
f.close

In [ ]:
data[0]['Label']['classifications'][0]['answer'][0]['value']

In [ ]:
data[0]['Labeled Data']

In [ ]:
punctuation = re.compile('[^\w\s]')
space = re.compile('  +')

clean_data = []
for i in range(len(data)):
    if bool(data[i]['Label']):


        sentence = sp(space.sub(' ', punctuation.sub('', data[i]['Labeled Data'])))
        




        if data[i]['Label']['classifications'][0]['answer'][0]['value'] == 'neutral':
            clean_data.append({'sentence': sentence, 'label': 0})
        elif data[i]['Label']['classifications'][0]['answer'][0]['value'] == 'negative':
            clean_data.append({'sentence': sentence, 'label': -1})
        else:
            clean_data.append({'sentence': sentence, 'label': 1})
    
clean_data[0]

In [ ]:
sentences_lemmaed = []
for sentence in clean_data:
    words = []
    for w in sentence['sentence']:
        words.append(w.lemma_.lower())
    sentences_lemmaed.append([" ".join(words), sentence['label']])
sentences_lemmaed[2]

In [ ]:
words = set()

for par in tqdm(sentences_lemmaed):
    for w in par[0].split():
        words.add(w)

words = list(words)
        
len(words)

In [ ]:
class_sentence_count = {-1: 0, 0: 0, 1: 0}

for par in tqdm(sentences_lemmaed):
    class_sentence_count[par[1]] += 1


In [ ]:
word_class_sentence_count = {}

for word in tqdm(words):
    word_class_sentence_count[word] = {-1: 0, 0: 0, 1: 0}
    for par in sentences_lemmaed:
        if word in par[0].split():
            word_class_sentence_count[word][par[1]] += 1


In [ ]:
word_sentence_count = {}
for k, v in word_class_sentence_count.items():
    word_sentence_count[k] = sum(v.values())

In [ ]:
total = len(sentences_lemmaed)
total

In [ ]:
pmi = {}

# p(w|c)
for word in tqdm(words):
    pmi[word] = {}
    
    if word_class_sentence_count[word][-1] != 0:
        pmi[word][-1] = math.log(
            (word_class_sentence_count[word][-1] / total) / 
            (class_sentence_count[-1] / total))
    else:
        pmi[word][-1] = math.log(
            (1 / total) / 
            (class_sentence_count[-1] / total))


    if word_class_sentence_count[word][0] != 0:
        pmi[word][0] = math.log(
            (word_class_sentence_count[word][0] / total) / 
            (class_sentence_count[0] / total))
    else:
        pmi[word][0] = math.log(
            (1 / total) / 
            (class_sentence_count[0] / total))

    
        
    if word_class_sentence_count[word][1] != 0:
        pmi[word][1] = math.log(
            (word_class_sentence_count[word][1] / total) / 
            (class_sentence_count[1] / total))
    else:
        pmi[word][1] = math.log(
            (1 / total) / 
            (class_sentence_count[1] / total))

In [ ]:
S = {}

for word, values in pmi.items():
    S[word] = values[1] - values[-1]

In [ ]:
s_df = pd.DataFrame.from_dict(S, orient='index')
s_df.columns = ['sent']
freq = pd.DataFrame.from_dict(word_sentence_count, orient='index')
freq.columns = ['freq']
s_df = s_df.join(freq)
s_df.sort_values('sent', ascending=False)

In [170]:
s_df.loc['lockdown']

sent    -0.489106
freq    34.000000
Name: lockdown, dtype: float64

In [185]:
vader_lexicon_file = './vader_lexicon.txt'
vader_lexicon = {}
with open(vader_lexicon_file, 'r') as f:
    for line in f.readlines():
        line = line.strip()
        vader_lexicon[line.split('\t')[0]] = float(line.split('\t')[1])
vader_words = list(vader_lexicon.keys())

s_new = s_df[~s_df.index.isin(vader_words)].copy()
s_com = s_df[s_df.index.isin(vader_words)].copy()

s_com['vader'] = s_com.apply(lambda x: vader_lexicon.get(x.name), axis=1)


In [186]:
s_com_filter = s_com[s_com['freq']>15].copy().reset_index()

In [187]:
px.scatter(s_com_filter, x='vader', y='sent', text='index', size='freq',
           labels={'vader': 'Vader score', 'sent': 'Customized score'}, 
           title='Comparison of Lexicon')

In [188]:
# nltk.download('stopwords')
# from nltk.corpus import stopwords

# s_new = [~s_new.index.isin(stopwords.words('english'))]
thres = 50
s_new_filter = s_new[s_new['freq']>thres].copy().reset_index()
s_new_filter

,index,sent,freq
0,for,0.029849,365
1,to,0.182009,895
2,high,-1.415869,56
3,china,-0.722721,58
4,while,0.004327,79
...,...,...,...
80,public,-0.329679,61
81,do,0.206815,93
82,a,-0.044726,645
83,include,0.006794,68


In [193]:
px.scatter(s_new_filter, x='freq', y='sent', text='index')

In [219]:
update_dict = {
    # existing
    "crisis": 0,
    "Crisis": 0,
    "positive": -1.5,
    "Positive": -1.5,


    "death": -1,
    "Death": -1,
    "die": -1.5,
    "Die": -1.5,
    "bad": -1,
    "Bad": -1,
    "fear": -1.5,
    "Fear": -1.5,
    
    "unemployment": -0.5,
    "emergency": 0,
    "fight": 1,
    "stop": 0.5,
    "loser": -0.5,
    "restriction": -0.5,
    "risk": 0,
    "cut": 0,
    "low":0,
    "number": -1,
    "care": -0.5,
    "increase": -0.5,
    "united": -0.5,
    "security": -1,

    "Unemployment": -0.5,
    "Emergency": 0,
    "Fight": 1,
    "Stop": 0.5,
    "Loser": -0.5,
    "Restriction": -0.5,
    "Risk": 0,
    "Cut": 0,
    "Low":0,
    "Number": -1,
    "Care": -0.5,
    "Increase": -0.5,
    "United": -0.5,
    "Security": -1,

    


    # new
    
    "pandemic": -0.5,
    "Pandemic": -0.5,
    "virus": -0.5,
    "Virus": -0.5,
    "covid19": -1,
    "Covid19": -1,
    "high": -1.5,
    "High": -1.5,
    "home": -1,
    "Home": -1,
    "report": -1,
    "Report": -1,
    "China": -0.5,
    "work": 1,
    "Work": 1,
    "government": 1,
    "Government": 1,
    "outbreak": -2,
    "Outbreak": -2,
    "lockdown": -0.5,
    "Lockdown": -0.5,
    "test": -0.5,
    "Test": -0.5,
    "business": 0.5,
    "Business": 0.5
    
}

In [220]:
s_df.loc['virus']

sent    -0.435039
freq    91.000000
Name: virus, dtype: float64

## Calculate Sentence Scores

In [221]:
sid = SentimentIntensityAnalyzer()
sid.lexicon.update(update_dict)

In [206]:
sentence_scores = []
labels = []
for i in range(len(data)):
    if bool(data[i]['Label']):
        sentence = data[i]['Labeled Data']
        sentence_scores.append(sid.polarity_scores(sentence)['compound'])
        label = data[i]['Label']['classifications'][0]['answer'][0]['value']
        if label == 'neutral':
            labels.append(0)
        elif label == 'negative':
            labels.append(-1)
        else:
            labels.append(1)


## Compute Accuracy Metrics

In [207]:
import numpy as np
from accuracy_metrics import *

def find_opt_metrics(predictions, gold_labels, lower_bound=-1, upper_bound=1, step=0.05):
    
    # opt_lower = -0.75
    # opt_upper = 0.75
    max_f1 = 0 
    opt_metrics = None
    for lower in np.arange(lower_bound, upper_bound, step):
        for upper in np.arange(upper_bound, lower, step*-1):
            metrics = AccuracyMetrics(predictions, gold_labels, neutral_upper=upper, neutral_lower=lower)
            if metrics.macro_f1 > max_f1:
                # opt_lower = lower
                # opt_upper = upper
                max_f1 = metrics.macro_f1
                opt_metrics = metrics
    
    return opt_metrics

In [208]:
opt_metrics = find_opt_metrics(sentence_scores, labels)

In [209]:
opt_neutral_lower = opt_metrics.neutral_upper
opt_neutral_upper = opt_metrics.neutral_upper

In [210]:
import csv
def write_opt_acc_metrics(accuracy_metrics, path):

    with open(path, mode='w') as csv_file:
        accuracy_writer = csv.writer(csv_file, delimiter=',', quotechar='"')
        accuracy_writer.writerow(list(accuracy_metrics.result_dict().keys()) + ['neutral_lower', 'neutral_upper'])

        this_met = accuracy_metrics
        accuracy_writer.writerow(list(this_met.result_dict().values()) + [str(this_met.neutral_lower), str(this_met.neutral_upper)])

In [211]:
write_opt_acc_metrics(opt_metrics, '../results/sentence_only_acc_metrics.csv')

## Make predictions on new sentences

In [226]:
f = open('./new_unlabelled_sentences.json')
new_sentences = json.load(f)
f.close

<function TextIOWrapper.close()>

In [231]:

output = []

for i in range(len(new_sentences)):
    score = sid.polarity_scores(new_sentences[i])['compound']
    if score < -0.05:
        output.append({'text': new_sentences[i], 'score': score, 'label': -1})
    elif score < 0.5:
        output.append({'text': new_sentences[i], 'score': score, 'label': 0})
    else:
        output.append({'text': new_sentences[i], 'score': score, 'label': 1})

In [234]:
with open("../results/new_sentence_scores.json", 'w') as outfile:
    json.dump(output, outfile)

## Score All Articles with Avg Sentence Scores

In [240]:
import pymysql.cursors
import pandas as pd
import datetime
import time

def convert_date(date_as_int):
    date_as_str = str(date_as_int)
    return datetime.date(int(date_as_str[0:4]), int(date_as_str[4:6]), int(date_as_str[6:8]))


connection = pymysql.connect(host="nlp2590.ckdech7dwvqp.us-east-1.rds.amazonaws.com",
    port=3310,
    user='nlp2590',
    password='lDABt9PoT7wJcdjljome',
    db='nlp2590')

valid_ids = []
articles = []
dates = []

try: 
    with connection.cursor() as cursor:
        sql = 'select news.RECORDID, news.body, gdelt_raw.DATE from news inner join gdelt_raw on news.RECORDID = gdelt_raw.RECORDID where news.body != ""; '
        print('starting database request')
        cursor.execute(sql)
        print('executed sql command')
        rows = cursor.fetchall()
        print('fetched data')
        for row in rows:
            valid_ids.append(row[0])
            articles.append(row[1])
            dates.append(convert_date(row[2]))
        print('finished reading data')
finally:
    connection.close()


starting database request
executed sql command
fetched data
finished reading data


In [245]:
def split_to_chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i: i+n]
        
valid_ids_chunks = list(split_to_chunks(valid_ids, 1000))
articles_chunks = list(split_to_chunks(articles, 1000))
dates_chunks = list(split_to_chunks(dates, 1000))

In [256]:

def get_sentences(article):
    article = str(article)
    article = article.replace('\xa0', '\n')
    sentences = re.split(r"\.|\?|\!", article)
    sentences = [sentence.replace("\\", "") for sentence in sentences]
    return sentences

def compute_sentence_scores(sentences):
    
    scores = []
    for sentence in sentences:
        scores.append(sid.polarity_scores(sentence)['compound'])
    return scores
def compute_article_sentiment(articles):
    scores = []
    for article in articles:
        sentences = get_sentences(article)
        sentence_scores = compute_sentence_scores(sentences)
        average_sentence = statistics.mean(sentence_scores)
        scores.append(average_sentence)
    return scores
    
def map_predictions(predictions, neutral_lower=-0.05, neutral_upper=0.5):
    print('starting to map predictions')
    mapped_predictions = []
    for index in range(len(predictions)):
        if predictions[index] <= neutral_lower:
            mapped_predictions.append(-1)
        elif predictions[index] <= neutral_upper:
            mapped_predictions.append(0)
        else:
            mapped_predictions.append(1)
    print('finished mapping predictions')
    return mapped_predictions

def write_to_csv(valid_ids, dates, sentiment_scores, predictions, path):
    print('starting to write csv')
    col_dict = {'RECORDID': valid_ids, 'dates': dates, 'sentiment_scores': sentiment_scores, 'predictions': predictions}
    df = pd.DataFrame(col_dict)
    df.to_csv(path, mode='a', index=False, header=False)
    print('finished writing csv')

In [246]:
for chunk_index in range(len(valid_ids_chunks)):

    program_start = time.time()

    print(f'Starting chunk {chunk_index}/{len(valid_ids_chunks)}. ')

    valid_ids = valid_ids_chunks[chunk_index]
    articles = articles_chunks[chunk_index]
    dates = dates_chunks[chunk_index]

    start_time = time.time()
    sentiment_scores =compute_article_sentiment(articles)
    predictions = map_predictions(sentiment_scores)

    write_to_csv(valid_ids, dates, sentiment_scores, predictions, '../results/all_articles_sentence_avg.csv')
    print(f'Finished chunk {chunk_index}/{len(valid_ids_chunks)}. Time: {time.time() - start_time}')

print(f'Total time: {time.time() - program_start}')



    



['20200101-1354',
 '20200101-7413',
 '20200101-35956',
 '20200101-41222',
 '20200102-35824',
 '20200102-59125',
 '20200102-67127',
 '20200102-70507',
 '20200102-75142',
 '20200103-8289',
 '20200103-8874',
 '20200103-14790',
 '20200103-16511',
 '20200103-22064',
 '20200103-33074',
 '20200103-48026',
 '20200103-50503',
 '20200103-71561',
 '20200103-72306',
 '20200104-7015',
 '20200104-14696',
 '20200104-38302',
 '20200104-48506',
 '20200105-32772',
 '20200105-40742',
 '20200105-43053',
 '20200106-6296',
 '20200106-26815',
 '20200106-27490',
 '20200106-39691',
 '20200106-43334',
 '20200106-50161',
 '20200106-51373',
 '20200106-77459',
 '20200106-86670',
 '20200107-3894',
 '20200107-7596',
 '20200107-16624',
 '20200107-26385',
 '20200107-27518',
 '20200107-27589',
 '20200107-29946',
 '20200107-30531',
 '20200107-36501',
 '20200107-42411',
 '20200107-44037',
 '20200107-62650',
 '20200107-67727',
 '20200107-81657',
 '20200107-85458',
 '20200108-29055',
 '20200108-52817',
 '20200108-58121',
 